In [ ]:
# Importing pytorch and other related libraries that will be required in this project

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models

from collections import OrderedDict
#import random, os
import time

from PIL import Image
import json

data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

# TODO: Define your transforms for the training, validation, and testing sets
training_transformations = transforms.Compose([transforms.RandomRotation(30),
                                               transforms.RandomResizedCrop(224),
                                               transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406],
                                                                    [0.229, 0.224, 0.225])])

testing_transformations = transforms.Compose([transforms.Resize(256),
                                              transforms.CenterCrop(224),
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.485, 0.456, 0.406],
                                                                   [0.229, 0.224, 0.225])])

validation_transformations = transforms.Compose([transforms.Resize(256),
                                                 transforms.CenterCrop(224),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                                      [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
training_dataset = datasets.ImageFolder(train_dir, transform=training_transformations)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transformations)
testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transformations)

# TODO: Using the image datasets and the trainforms, define the dataloaders
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64, shuffle=True)
testing_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=64, shuffle=True)


with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Load a pretrained network
# According to the guidelines: The VGG network is easier to use
model = models.vgg16(pretrained = True)
model

# Freeze model parameters to avoid backpropagation
for param in model.parameters():
    param.requires_grad = False

# Since I have been offered GPU for a plenty amount of time
# I'll define my device as CUDA to leverage the GPU power
device = torch.device('cuda')

# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
classifier = nn.Sequential(OrderedDict([
    ('dropout', nn.Dropout(0.2)),
    ('fc1', nn.Linear(25088, 512)),
    ('relu1', nn.ReLU()),
    ('fc2', nn.Linear(512, 256)),
    ('relu2', nn.ReLU()),
    ('fc3', nn.Linear(256, 128)),
    ('relu3', nn.ReLU()),
    ('output', nn.Linear(128, 102)),
    ('log_softmax', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier
model

# At this point, I'm going to move my model to CUDA so that it can train using GPU
model = model.to(device)

# Define loss function, learning rate, optimizer, epochs, print_every, steps
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
epochs = 5
print_every = 10
steps = 0

training_losses = []
validation_losses = []

print('Training has started')

for epoch in range(epochs):
    running_loss = 0

    for ii, (training_images, training_labels) in enumerate(training_loader):
        # Increase the steps by 1
        steps += 1

        # get data(images and labels) to GPU, since I have it available
        training_images, training_labels = training_images.to(device), training_labels.to(device)

        # Setting all gradients for each batch to 0
        optimizer.zero_grad()

        # Forward passes
        outputs = model.forward(training_images)
        loss = criterion(outputs, training_labels)

        # Backward passes
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        training_losses.append((running_loss / len(training_loader)))

        if steps % print_every == 0:
            model.eval()
            validation_loss = 0
            accuracy = 0

            with torch.no_grad():
                for ii, (validation_images, validation_labels) in enumerate(validation_loader):
                    validation_images, validation_labels = validation_images.to(device), validation_labels.to(device)
                    log_ps = model(validation_images)
                    loss = criterion(log_ps, validation_labels)
                    validation_loss += loss.item()

                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equality = top_class == validation_labels.view(*top_class.shape)
                    accuracy += torch.mean(equality.type(torch.FloatTensor)).item()

            # Calculating the average validation loss and accuracy
            validation_loss /= len(validation_loader)
            accuracy /= len(validation_loader)

            print(f"Epoch: {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/len(training_loader):.3f}.. "
                  f"Test loss: {validation_loss:.3f}.. "
                  f"Test accuracy: {accuracy:.3f}")
            running_loss = 0
            model.train()
print("\nTraining has ended:)")

# TODO: Do validation on the test set

accuracy = 0
print("Testing has started...")

with torch.no_grad():
    for ii, (testing_images, testing_labels) in enumerate(testing_loader):
        testing_images, testing_labels = testing_images.to(device), testing_labels.to(device)
        log_ps = model(testing_images)

        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equality = top_class == testing_labels.view(*top_class.shape)
        accuracy += torch.mean(equality.type(torch.FloatTensor)).item()

accuracy /= len(testing_loader) * 100

print(f"After testing, the accuracy of model is: {accuracy:.3f}%")
print("Testing has ended!")

# Displaying the training and validation loss on a plot
plt.plot(training_losses, label='Training loss')
plt.plot(validation_losses, label='Testing loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend();
plt.show()

# TODO: Save the checkpoint 
checkpoint = {
    'input_size': 25088,
    'output_size': 102,
    'hidden_size': [512, 256, 128],
    'dropout': 0.2,
    'learning_rate': 0.001,
    'model_architecture': 'vgg16',
    'model_state_dict': model.state_dict(),
    'class_to_idx': training_dataset.class_to_idx,
    'epochs': epochs,
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(checkpoint, 'checkpoint.pth')

# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(pth_fname):
    checkpoint = torch.load(pth_fname)
    model_architecture = checkpoint['model_architecture']

    # Loading pretrained model
    model = getattr(models, model_architecture)(pretrained=False)

    # Loading the classifier and model state_dict
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['model_state_dict'])

    # Initializing the optimizer with the correct learning rate and loading the optimizer state_dict
    optimizer = optim.Adam(model.classifier.parameters(), lr=checkpoint['learning_rate'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # Checkpoint values
    input_size = checkpoint['input_size']
    hidden_size = checkpoint['hidden_size']
    output_size = checkpoint['output_size']
    class_to_idx = checkpoint['class_to_idx']
    epochs = checkpoint['epochs']

    return model, optimizer, input_size, hidden_size, output_size, class_to_idx, epochs

# Load the checkpoint
model, optimizer, input_size, hidden_size, output_size, class_to_idx, epochs = load_checkpoint('checkpoint.pth')

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # TODO: Process a PIL image for use in a PyTorch model
    # Defining transfomations to resize, crop out the center, normalize, and converting the image to a pytorch tensor 
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])

    image = preprocess(image)
    return image

def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)

    ax.imshow(image)

    return ax

image_path = 'flowers/test/100/image_07902.jpg'

# Display original image
with Image.open(image_path) as image:
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

# Display preprocessed image
with Image.open(image_path) as image:
    processed_image = process_image(image)
    imshow(processed_image, title='Processed Image')
    plt.show()

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # TODO: Implement the code to predict the class from an image file

    # Preprocess the image
    # adding batch dimension since networks take input as batches
    # https://pytorch.org/docs/stable/tensors.html#torch.Tensor
    img_tensor = process_image(image_path).unsqueeze(0).to(device)

    # Set the model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        output = model(img_tensor)
        probabilities, indices = torch.topk(torch.exp(output), topk)

    # Convert indices to class labels
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    classes = [idx_to_class[idx.item()] for idx in indices[0]]

    return probabilities, classes

image_path = 'flowers/test/101/image_07952.jpg'
probs, classes = predict(image_path, model)
print(probs)
print(classes)